# Weekly Core Metric Analysis

This notebook provides pre-processing, analysis, and visualization on weekly ATRT data provided by Cerner (Armand Kok) in a .csv dataset that includes daily (aggregated) timer data, transaction counts, etc... for a 90-day lookback period. 

10/26:
- **Business questions**:
    - Which timers are negatively affecting ATRT? 
    - What is the makeup of ATRT for a given period, and how has that changed over the previous periods? 
    - When did a shift begin for a particular timer? 
    - What are the smoothed trends for our core metrics overall?  
    - What is our ATRT for relative dates (month to date, past 30/60/90, etc). 

**To-do**
- [ ] consider moving to 120 day lookback period for control charts. 
- [ ] Joining most recent week's data: 
    - MySQL/postgres database that will index based on timer, date, application. 
    - run import job on .csv every week (Friday/Monday) to update with current week's data.

In [30]:
# Import libraries/packages
import csv
import pandas as pd
import datetime
import numpy as np
import os
from datetime import datetime 
from pandas.tseries.offsets import BDay
from scipy.stats.mstats import winsorize
#import statsmodels.api as sm
#import seaborn as sns
#import matplotlib.pyplot as plt

### Import dataset and make sure it's working. 
**To-do**:
- 10/25 automate dataset generation. Use case for hook into Vertica: Armand is currently manually giving this ds weekly. 
- 10/25 find best place to store to capture history

In [31]:
#client_atrt = pd.read_csv('Z:\IUH\khickman1\Datasets\client_atrt10.15.18.csv')
client_atrt = pd.read_csv('C:\\Users\\khickman1\\Desktop\\client_atrt10.29.csv')
#client_atrt = client_atrt[['dt', 'application_name', 'Timer Subtimer Name', 'ATRT']]
client_atrt.head()
len(client_atrt) #should be around 650-680k rows

676399

## Preprocessing

 - Dates
 - Filtering
 - Outlier removal
 - moving average calculation
 - window calculation
 - if/else statement based on window 
 - calculate UCL/LCL
 - feature selection

**To-Do**
 - [ ] remove subtimer name? 
 
### Date column transformation

In [56]:
atrt_df = pd.DataFrame(client_atrt)
atrt_df['dt'] = pd.to_datetime(atrt_df['dt'])
print(atrt_df['dt'][0:10])
atrt_df.head()
# check to make sure the 'dt' column is type:datetime64

0   2018-10-01
1   2018-10-29
2   2018-08-15
3   2018-08-28
4   2018-09-11
5   2018-10-19
6   2018-08-20
7   2018-10-17
8   2018-09-13
9   2018-08-08
Name: dt, dtype: datetime64[ns]


ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
0  0.04                         0.00%                       100.00%   
1  0.41                         0.00%                       100.00%   
2  0.65                         0.00%                       100.00%   
3  0.15                         0.00%                       100.00%   
4  0.38                         0.00%                       100.00%   

                                 Timer Subtimer Name  Number of Records  \
0  USR:SRG SNDOCDISP DISCONTINUING SEGMENT (BMH P...                  1   
1     USR: PDOC NAVIGATOR BAND CLICK (NICU FINNEGAN)                  1   
2        USR: ICU-OPENING-BAND (PCA ADLS AND SAFETY)                  1   
3  USR: PDOC NAVIGATOR DOC SET CLICK (PICU RISK A...                  1   
4  USR:MPG.DOCUMENTATION_HPI.O1 - LOAD COMPONENT ...                  1   

  application_name         dt      strt                  subtimername  \
0   SNSURGINET.EXE 2018-10-01  0.003983       BMH PACU PHASE 1 DELAYS   
1       POWERCHART 2018-10-29  0.087445                 NICU FINNEGAN   
2         FIRSTNET 2018-08-15  0.075630           PCA ADLS AND SAFETY   
3       POWERCHART 2018-08-28  0.096875          PICU RISK ASSESSMENT   
4       POWERCHART 2018-09-11  0.311815  VB_WORKFLOWAMBPEDSCARDIOLOGY   

                                       timername  total_elapsed  transAbove5  \
0        USR:SRG SNDOCDISP DISCONTINUING SEGMENT       0.800640            0   
1                 USR: PDOC NAVIGATOR BAND CLICK      13.028515            0   
2                          USR: ICU-OPENING-BAND       4.530593            0   
3              USR: PDOC NAVIGATOR DOC SET CLICK      11.668000            0   
4  USR:MPG.DOCUMENTATION_HPI.O1 - LOAD COMPONENT      36.360582            0   

   transUnder2  transaction_cnt     isBDay  
0           21               21 2018-10-01  
1           32               32 2018-10-29  
2            7                7 2018-08-15  
3           79               79 2018-08-28  
4           95               95 2018-09-11

### Remove non-working days.  

Currently only removing weekends. 

**To-do**
- remove federal/bank holidays?  E.g. sept 3 was labor day. 
- clean up is-bday column via subsetting or boolean

In [57]:
isBusinessDay = BDay().onOffset
match_series = atrt_df['dt'].map(isBusinessDay)
atrt_df['isBDay'] = atrt_df['dt'][match_series]
atrt_df

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
0       0.04                         0.00%                       100.00%   
1       0.41                         0.00%                       100.00%   
2       0.65                         0.00%                       100.00%   
3       0.15                         0.00%                       100.00%   
4       0.38                         0.00%                       100.00%   
5       0.21                         0.00%                       100.00%   
6       0.53                         0.00%                       100.00%   
7       1.15                         0.00%                       100.00%   
8       0.61                         0.00%                       100.00%   
9       0.29                         0.00%                       100.00%   
10      0.08                         0.00%                       100.00%   
11      0.69                         0.00%                       100.00%   
12      0.31                         0.00%                       100.00%   
13      2.27                         0.66%                        35.77%   
14      0.04                         0.00%                       100.00%   
15      0.08                         0.00%                       100.00%   
16      0.10                         0.00%                       100.00%   
17      0.08                         0.00%                       100.00%   
18      0.05                         0.00%                       100.00%   
19      0.06                         0.00%                       100.00%   
20      0.04                         0.00%                       100.00%   
21      0.04                         0.00%                       100.00%   
22      0.63                         0.00%                        96.30%   
23      0.04                         0.00%                       100.00%   
24      0.23                         0.00%                       100.00%   
25      0.04                         0.00%                       100.00%   
26      0.24                         0.00%                        99.87%   
27      0.46                         0.00%                       100.00%   
28      0.15                         0.00%                       100.00%   
29      0.14                         0.00%                       100.00%   
...      ...                           ...                           ...   
676369  0.29                         0.00%                       100.00%   
676370  0.04                         0.00%                       100.00%   
676371  0.31                         0.00%                       100.00%   
676372  0.12                         0.00%                       100.00%   
676373  0.52                         0.00%                       100.00%   
676374  0.61                         0.03%                        99.78%   
676375  0.68                         0.00%                       100.00%   
676376  1.25                         0.00%                        76.74%   
676377  0.12                         0.00%                       100.00%   
676378  0.30                         0.00%                       100.00%   
676379  0.04                         0.00%                       100.00%   
676380  0.20                         0.00%                       100.00%   
676381  0.07                         0.00%                       100.00%   
676382  0.56                         0.00%                       100.00%   
676383  0.80                         0.00%                       100.00%   
676384  0.06                         0.00%                       100.00%   
676385  0.14                         0.00%                       100.00%   
676386  0.45                         0.00%                       100.00%   
676387  0.08                         0.00%                       100.00%   
676388  0.61                         0.00%                       100.00%   
676389  1.41                         0.00%              

### Filter dataset by application - e.g. Powerchart, firstnet, surginet.
(Actually apply the weekend/weekday filter here as well). 

**to-do**
- [ ] are there differences between timers in the different applications? 
- [X]clean up the workflow to do one step at a time

In [58]:
atrt_df_PC = atrt_df[atrt_df['application_name']== 'POWERCHART']
atrt_df_PC = atrt_df_PC[atrt_df_PC.isBDay.notnull()]
atrt_df_PC

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
1       0.41                         0.00%                       100.00%   
3       0.15                         0.00%                       100.00%   
4       0.38                         0.00%                       100.00%   
5       0.21                         0.00%                       100.00%   
13      2.27                         0.66%                        35.77%   
17      0.08                         0.00%                       100.00%   
22      0.63                         0.00%                        96.30%   
26      0.24                         0.00%                        99.87%   
29      0.14                         0.00%                       100.00%   
30      0.25                         0.00%                       100.00%   
31      0.21                         0.00%                       100.00%   
35      0.81                         0.02%                        99.87%   
36      0.29                         0.00%                       100.00%   
43      0.32                         0.00%                       100.00%   
48      0.25                         0.00%                       100.00%   
50      0.30                         0.00%                       100.00%   
52      2.52                         0.00%                         0.00%   
54      1.20                         1.19%                        88.76%   
58      1.90                         5.10%                        64.48%   
61      0.39                         0.00%                       100.00%   
65      0.42                         0.00%                       100.00%   
66      0.50                         0.00%                        97.30%   
68      0.29                         0.15%                        98.67%   
69      1.50                         7.14%                        85.71%   
72      0.37                         0.00%                        99.63%   
73      0.00                         0.00%                       100.00%   
76      0.91                         0.00%                        96.67%   
77      0.85                         0.80%                        87.20%   
79      1.30                         1.20%                        90.36%   
84      0.03                         0.00%                       100.00%   
...      ...                           ...                           ...   
676280  0.22                         0.00%                        99.96%   
676283  0.23                         0.00%                       100.00%   
676285  2.94                         0.00%                         0.00%   
676293  0.91                         0.00%                       100.00%   
676297  0.77                         0.00%                       100.00%   
676310  0.93                         1.85%                        92.59%   
676318  0.26                         0.00%                        99.87%   
676320  0.83                         2.28%                        95.19%   
676321  0.95                         1.95%                        91.56%   
676322  0.48                         0.47%                        97.32%   
676331  0.59                         0.00%                       100.00%   
676336  1.38                         0.00%                        86.73%   
676340  0.21                         0.02%                        99.90%   
676349  0.36                         0.12%                        98.13%   
676351  1.21                         3.09%                        92.78%   
676353  0.12                         0.00%                        99.85%   
676367  1.13                         0.00%                        86.67%   
676371  0.31                         0.00%                       100.00%   
676372  0.12                         0.00%                       100.00%   
676374  0.61                         0.03%                        99.78%   
676377  0.12                         0.00%              

## Anomaly detection/Handling

#### Applying quantiles:

Apply 5th and 95th quantiles to each timer for the entire 90 day period. 
Then we can lookup the values using ```loc``` and ```filter``` 


**To-do**
- Investigate outliers/anomalies by timer. Replace with mean/remove altogether. 
- Use KNN? 
- Complete - remove outliers by timer. 

Sample outlier removal calculation by group. 
```atrt_df_PC[np.abs(atrt_df_PC.ATRT-atrt_df_PC.ATRT.mean()) <= (3*atrt_df_PC.ATRT.std())]```

Takes the absolute value of standardized value for each datapoint, then removes it if it's less than 3 standard deviations from the mean. 
- I need to do this by group! 
- Group first, then pass in group calculation. 

In [59]:
#find outliers
outliers = atrt_df_PC.groupby(["Timer Subtimer Name"])['ATRT'].quantile([0.05, 0.95]).unstack(level=1)
#filter outliers
atrt_df_PC = atrt_df_PC.loc[((outliers.loc[atrt_df_PC['Timer Subtimer Name'], .05] < atrt_df_PC.ATRT.values) & (atrt_df_PC.ATRT.values < outliers.loc[atrt_df_PC['Timer Subtimer Name'], .95])).values]


### Rolling average notes
Lambda function for applying rolling average to a group: 
The groupby statement can be in an earlier variable, but I've chosen to include it here. 

**To-do**
- [ ] Find out exactly what reset_index does - reset the average calculation to the beginning of each group? 
- [ ] Remove outliers/anomalies by timer. Replace with mean/remove altogether. 
- [ ] Use KNN for clustering timers...analyze what timers that have shifted up have in common. 
- [ ] Establish control chart
    - UCL = .75 quartile + 1.5x IQR
    - LCL = .25 quartile - 1.5x IQR
    - mean = 60-day mean
    - median = 60-day median (use with outliers included)
```
atrt_df_PC.groupby(['Timer Subtimer Name', 'dt'])['ATRT'].rolling(30).mean().reset_index(0,drop=True)
```
Pass into dataset feature with ```atrt_df_PC['mavg30'] = (above statement)```

```atrt_df_PC['mavg30'] = atrt_df_PC.groupby(['Timer Subtimer Name', 'dt'])['ATRT'].rolling(30).mean().reset_index(0,drop=True)```

In [60]:
#Setting index appears to cause issues when outputting to csv: 
#atrt_df_PC = atrt_df_PC.set_index(['Timer Subtimer Name', 'dt'])

atrt_df_PC = atrt_df_PC.sort_values(['Timer Subtimer Name', 'dt'])
atrt_df_PC['mavg30'] = atrt_df_PC.groupby('Timer Subtimer Name')['ATRT'].apply(lambda x:x.rolling(center=False,window=30).mean())

#check to make sure that everything looks correct: 
atrt_df_PC


ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
642711  0.14                         0.02%                        99.98%   
280854  0.14                         0.02%                        99.98%   
447545  0.15                         0.00%                       100.00%   
376550  0.16                         0.00%                        99.98%   
305295  0.15                         0.00%                       100.00%   
98917   0.15                         0.00%                        99.98%   
426129  0.16                         0.00%                        99.98%   
347498  0.16                         0.00%                        99.98%   
8354    0.16                         0.00%                        99.96%   
661101  0.15                         0.00%                       100.00%   
591884  0.15                         0.00%                       100.00%   
424275  0.15                         0.02%                        99.98%   
389254  0.15                         0.00%                       100.00%   
137556  0.16                         0.00%                        99.97%   
438317  0.16                         0.00%                        99.97%   
290374  0.15                         0.00%                       100.00%   
68971   0.16                         0.00%                        99.95%   
163210  0.15                         0.00%                        99.98%   
201982  0.15                         0.00%                       100.00%   
141399  0.15                         0.00%                        99.98%   
629705  0.15                         0.00%                       100.00%   
292636  0.15                         0.00%                        99.98%   
577847  0.15                         0.00%                        99.98%   
33072   0.14                         0.00%                       100.00%   
479463  0.15                         0.00%                       100.00%   
245482  0.14                         0.00%                       100.00%   
106866  0.15                         0.00%                        99.96%   
341563  0.15                         0.00%                       100.00%   
271075  0.15                         0.00%                       100.00%   
673451  0.15                         0.00%                       100.00%   
...      ...                           ...                           ...   
472877  4.12                        45.59%                        50.00%   
177931  4.96                        56.04%                        37.36%   
274947  4.18                        43.04%                        46.84%   
349579  4.72                        53.85%                        44.87%   
622270  4.49                        48.28%                        50.00%   
592019  4.95                        53.42%                        41.10%   
650866  4.76                        52.33%                        38.37%   
509058  4.38                        48.72%                        46.15%   
522980  4.60                        53.03%                        37.88%   
175509  4.29                        46.74%                        43.48%   
8468    4.75                        52.63%                        44.74%   
81227   4.17                        48.78%                        47.56%   
657999  3.89                        41.57%                        53.93%   
98703   4.53                        51.28%                        43.59%   
491276  3.89                        43.30%                        50.52%   
463314  4.69                        48.28%                        46.55%   
548273  4.75                        52.94%                        43.14%   
274934  4.84                        55.00%                        40.00%   
623825  4.53                        51.47%                        39.71%   
667601  4.06                        46.75%                        50.65%   
61472   4.53                        55.41%              

### Windows

Add window - 7/8 day window using pandas shift(). 

Difference between Trend and Shift
- trend = "7 days in a row"
- shift = "8 days above median"

**to-do**
- [x] start with shift/window. 
- [ ] establish how to identify trends and shifts using elif statements.

**questions**
- what happens when I compare something with a NaN value? - e.g. I compare days 1-7 with NaN? 
- do I need to drop rows with NaN, or return a null value? 

In [61]:
# dataframe var is atrt_df_PC

atrt_df_PC['t-1'] = atrt_df_PC['ATRT'].shift(1)
atrt_df_PC['t-2'] = atrt_df_PC['ATRT'].shift(2)
atrt_df_PC['t-3'] = atrt_df_PC['ATRT'].shift(3)
atrt_df_PC['t-4'] = atrt_df_PC['ATRT'].shift(4)
atrt_df_PC['t-5'] = atrt_df_PC['ATRT'].shift(5)
atrt_df_PC['t-6'] = atrt_df_PC['ATRT'].shift(6)
atrt_df_PC['t-7'] = atrt_df_PC['ATRT'].shift(7)
atrt_df_PC['t-8'] = atrt_df_PC['ATRT'].shift(8)

df_pc = atrt_df_PC
df_pc.head()

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
642711  0.14                         0.02%                        99.98%   
280854  0.14                         0.02%                        99.98%   
447545  0.15                         0.00%                       100.00%   
376550  0.16                         0.00%                        99.98%   
305295  0.15                         0.00%                       100.00%   

               Timer Subtimer Name  Number of Records application_name  \
642711  DMSM_GETMEDIACONTENT (N/A)                  1       POWERCHART   
280854  DMSM_GETMEDIACONTENT (N/A)                  1       POWERCHART   
447545  DMSM_GETMEDIACONTENT (N/A)                  1       POWERCHART   
376550  DMSM_GETMEDIACONTENT (N/A)                  1       POWERCHART   
305295  DMSM_GETMEDIACONTENT (N/A)                  1       POWERCHART   

               dt      strt subtimername             timername ...   \
642711 2018-07-31  0.273690          NaN  DMSM_GETMEDIACONTENT ...    
280854 2018-08-01  0.143897          NaN  DMSM_GETMEDIACONTENT ...    
447545 2018-08-02  0.074053          NaN  DMSM_GETMEDIACONTENT ...    
376550 2018-08-03  0.086916          NaN  DMSM_GETMEDIACONTENT ...    
305295 2018-08-06  0.080803          NaN  DMSM_GETMEDIACONTENT ...    

           isBDay  mavg30   t-1   t-2   t-3   t-4  t-5  t-6  t-7  t-8  
642711 2018-07-31     NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  
280854 2018-08-01     NaN  0.14   NaN   NaN   NaN  NaN  NaN  NaN  NaN  
447545 2018-08-02     NaN  0.14  0.14   NaN   NaN  NaN  NaN  NaN  NaN  
376550 2018-08-03     NaN  0.15  0.14  0.14   NaN  NaN  NaN  NaN  NaN  
305295 2018-08-06     NaN  0.16  0.15  0.14  0.14  NaN  NaN  NaN  NaN  

[5 rows x 24 columns]

## Control Charts
Set upper and lower control limits, 60-day mean, signal detection. 
- Upper and lower control limits. 
    - Upper OUTLIER = .75 qt + 1.5x IQR??
    - [x] Upper CONTROL = .75 qt 
    - [ ] Lower Outlier = .25 qt - 1.5x IQR
    - [x] Lower control = .25 qt 
    - [x] Obtain IQR (on 60 day mean) for each timer. 
    - [ ] Multiply that by 1.5, set as variable
    - [x] Then calculate UCL and LCL for each timer. 
    
- Shift detection:
    - When t-n 8 of the previous N days above/below the median/mean
    - elif statement? 
   
- Trend detection:
    - When t-n (7) days in a row are higher than the previous day.
    
**To-Do**

- [x] interquartile range - loc lookup to ds? running calc? pandas has quartile built into running package
- [x] drop NA values after creating all relevant metrics. 
- [ ] plotting control charts for timer. 


**Questions**

Does it make sense to aggregate by week for smoother trends and less volatility? 
Which timers gave consistent up/down trend/shift signals for that week? Month? 
Which timers were outside the control limits more than N times during the previous period? 

### Quantiles

In [81]:
df_pc['LCL_30'] = df_pc['ATRT'].rolling(30).quantile(.25, interpolation='lower')
df_pc['UCL_30'] = df_pc['ATRT'].rolling(30).quantile(.75, interpolation='higher')
df_pc['var_30'] = df_pc['ATRT'].rolling(30).var(ddof=1)

#IQR for future outlier removal
#df_pc['IQR'] = df_pc['.75'] - df_pc['.25']

df_pc

C:\Users\khickman1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\khickman1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\khickman1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
673451  0.15                         0.00%                       100.00%   
261648  0.16                         0.00%                        99.98%   
48954   0.15                         0.00%                        99.96%   
394703  0.15                         0.03%                        99.97%   
332814  0.14                         0.03%                        99.95%   
641619  0.15                         0.00%                        99.95%   
383603  0.15                         0.00%                        99.98%   
399610  0.14                         0.00%                       100.00%   
392869  0.14                         0.00%                       100.00%   
571543  0.14                         0.00%                       100.00%   
2011    0.14                         0.00%                        99.98%   
438295  0.14                         0.00%                       100.00%   
240176  0.14                         0.00%                       100.00%   
255799  0.14                         0.00%                       100.00%   
491807  0.14                         0.00%                       100.00%   
445412  0.14                         0.00%                       100.00%   
545608  0.70                         0.00%                       100.00%   
400496  1.07                         0.00%                        66.67%   
662791  0.37                         0.00%                       100.00%   
526067  0.41                         0.00%                       100.00%   
371789  0.63                         0.00%                       100.00%   
303224  0.53                         0.00%                       100.00%   
388489  0.84                         0.00%                       100.00%   
471875  0.47                         0.00%                       100.00%   
480810  0.58                         0.00%                       100.00%   
338726  0.81                         0.00%                       100.00%   
547527  0.67                         0.00%                       100.00%   
350641  0.72                         0.00%                       100.00%   
126451  0.36                         0.00%                       100.00%   
430226  0.64                         0.00%                       100.00%   
...      ...                           ...                           ...   
602106  0.67                         0.00%                       100.00%   
527066  0.78                         0.00%                       100.00%   
274947  4.18                        43.04%                        46.84%   
349579  4.72                        53.85%                        44.87%   
622270  4.49                        48.28%                        50.00%   
592019  4.95                        53.42%                        41.10%   
650866  4.76                        52.33%                        38.37%   
509058  4.38                        48.72%                        46.15%   
522980  4.60                        53.03%                        37.88%   
175509  4.29                        46.74%                        43.48%   
8468    4.75                        52.63%                        44.74%   
81227   4.17                        48.78%                        47.56%   
657999  3.89                        41.57%                        53.93%   
98703   4.53                        51.28%                        43.59%   
491276  3.89                        43.30%                        50.52%   
463314  4.69                        48.28%                        46.55%   
548273  4.75                        52.94%                        43.14%   
274934  4.84                        55.00%                        40.00%   
623825  4.53                        51.47%                        39.71%   
667601  4.06                        46.75%                        50.65%   
61472   4.53                        55.41%              

### Remove missing values
Removing any values without the 30 day moving average.
Use ```np.isfinite``` instead of dropping na values. 

**ToDo**
    
    [ ] should we instead just use a dummy var if any value isna? 

In [84]:
df_pc = df_pc[np.isfinite(df_pc['mavg30'])]
df_pc

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
673451  0.15                         0.00%                       100.00%   
261648  0.16                         0.00%                        99.98%   
48954   0.15                         0.00%                        99.96%   
394703  0.15                         0.03%                        99.97%   
332814  0.14                         0.03%                        99.95%   
641619  0.15                         0.00%                        99.95%   
383603  0.15                         0.00%                        99.98%   
399610  0.14                         0.00%                       100.00%   
392869  0.14                         0.00%                       100.00%   
571543  0.14                         0.00%                       100.00%   
2011    0.14                         0.00%                        99.98%   
438295  0.14                         0.00%                       100.00%   
240176  0.14                         0.00%                       100.00%   
255799  0.14                         0.00%                       100.00%   
491807  0.14                         0.00%                       100.00%   
445412  0.14                         0.00%                       100.00%   
545608  0.70                         0.00%                       100.00%   
400496  1.07                         0.00%                        66.67%   
662791  0.37                         0.00%                       100.00%   
526067  0.41                         0.00%                       100.00%   
371789  0.63                         0.00%                       100.00%   
303224  0.53                         0.00%                       100.00%   
388489  0.84                         0.00%                       100.00%   
471875  0.47                         0.00%                       100.00%   
480810  0.58                         0.00%                       100.00%   
338726  0.81                         0.00%                       100.00%   
547527  0.67                         0.00%                       100.00%   
350641  0.72                         0.00%                       100.00%   
126451  0.36                         0.00%                       100.00%   
430226  0.64                         0.00%                       100.00%   
...      ...                           ...                           ...   
602106  0.67                         0.00%                       100.00%   
527066  0.78                         0.00%                       100.00%   
274947  4.18                        43.04%                        46.84%   
349579  4.72                        53.85%                        44.87%   
622270  4.49                        48.28%                        50.00%   
592019  4.95                        53.42%                        41.10%   
650866  4.76                        52.33%                        38.37%   
509058  4.38                        48.72%                        46.15%   
522980  4.60                        53.03%                        37.88%   
175509  4.29                        46.74%                        43.48%   
8468    4.75                        52.63%                        44.74%   
81227   4.17                        48.78%                        47.56%   
657999  3.89                        41.57%                        53.93%   
98703   4.53                        51.28%                        43.59%   
491276  3.89                        43.30%                        50.52%   
463314  4.69                        48.28%                        46.55%   
548273  4.75                        52.94%                        43.14%   
274934  4.84                        55.00%                        40.00%   
623825  4.53                        51.47%                        39.71%   
667601  4.06                        46.75%                        50.65%   
61472   4.53                        55.41%              

### Signal detection: 

- when did the signal start? 
- what is the signal up/down shift or trend? 
- include "magnitude of change"
- how about cumulative sum? 
- abs(t1-t2) + abs(t2-t3)


**Questions**

- How useful is identifying a spike if we don't have daily data? 

In [95]:
df_pc_test = df_pc
df_pc_test

def signal_detect(df):

    if (df['t-8'] >= df['mavg30'] and (df['t-7'] >= df['mavg30'])):
        return 'Red'
    #elif (df['trigger2'] <= df['score'] < df['trigger3']) and (df['height'] < 8):
    #    return 'Yellow'
    #elif (df['trigger3'] <= df['score']) and (df['height'] < 8):
    #    return 'Orange'
    #elif (df['height'] > 8):
    #    return np.nan

df_pc_test['signal'] = df_pc_test.apply(signal_detect, axis = 1)
df_pc_test

ATRT % of Transactions > 5 seconds % of Transactions < 2 seconds  \
673451  0.15                         0.00%                       100.00%   
261648  0.16                         0.00%                        99.98%   
48954   0.15                         0.00%                        99.96%   
394703  0.15                         0.03%                        99.97%   
332814  0.14                         0.03%                        99.95%   
641619  0.15                         0.00%                        99.95%   
383603  0.15                         0.00%                        99.98%   
399610  0.14                         0.00%                       100.00%   
392869  0.14                         0.00%                       100.00%   
571543  0.14                         0.00%                       100.00%   
2011    0.14                         0.00%                        99.98%   
438295  0.14                         0.00%                       100.00%   
240176  0.14                         0.00%                       100.00%   
255799  0.14                         0.00%                       100.00%   
491807  0.14                         0.00%                       100.00%   
445412  0.14                         0.00%                       100.00%   
545608  0.70                         0.00%                       100.00%   
400496  1.07                         0.00%                        66.67%   
662791  0.37                         0.00%                       100.00%   
526067  0.41                         0.00%                       100.00%   
371789  0.63                         0.00%                       100.00%   
303224  0.53                         0.00%                       100.00%   
388489  0.84                         0.00%                       100.00%   
471875  0.47                         0.00%                       100.00%   
480810  0.58                         0.00%                       100.00%   
338726  0.81                         0.00%                       100.00%   
547527  0.67                         0.00%                       100.00%   
350641  0.72                         0.00%                       100.00%   
126451  0.36                         0.00%                       100.00%   
430226  0.64                         0.00%                       100.00%   
...      ...                           ...                           ...   
602106  0.67                         0.00%                       100.00%   
527066  0.78                         0.00%                       100.00%   
274947  4.18                        43.04%                        46.84%   
349579  4.72                        53.85%                        44.87%   
622270  4.49                        48.28%                        50.00%   
592019  4.95                        53.42%                        41.10%   
650866  4.76                        52.33%                        38.37%   
509058  4.38                        48.72%                        46.15%   
522980  4.60                        53.03%                        37.88%   
175509  4.29                        46.74%                        43.48%   
8468    4.75                        52.63%                        44.74%   
81227   4.17                        48.78%                        47.56%   
657999  3.89                        41.57%                        53.93%   
98703   4.53                        51.28%                        43.59%   
491276  3.89                        43.30%                        50.52%   
463314  4.69                        48.28%                        46.55%   
548273  4.75                        52.94%                        43.14%   
274934  4.84                        55.00%                        40.00%   
623825  4.53                        51.47%                        39.71%   
667601  4.06                        46.75%                        50.65%   
61472   4.53                        55.41%              

### Investigate the data on a weekly basis
Counts
Sums
Averages
Variance

### Write out to csv:

In [81]:
atrt_df_PC.to_csv('C:\\Users\\khickman1\\Desktop\\PCMavg30.csv', sep=',')

### Stuff I learned: 

- Moving average example
- Subsetting
- Date and column filtering
- Index by both date and by timer/subtimer using pandas 'set index'
- Indexing
- Sorting
- Get unique values
- grouping
- filtering outliers (clipping, trimming, winsorizing)
- moving data

```
open_chart = atrt_df['Timer Subtimer Name' == "USR:PWR-OPEN CHART (DISCERNRPT/DISCERNPCTAB.DLL)"]
open_chart = atrt_df[atrt_df['Timer Subtimer Name'] == 'USR:PWR-OPEN CHART (DISCERNRPT/DISCERNPCTAB.DLL)']
open_chart = open_chart[open_chart['application_name']== 'POWERCHART']
open_chart_post = open_chart[(open_chart['dt'] > '2018-07-01') & (open_chart['dt'] < '2018-09-30')]
open_chart = open_chart.set_index(['Timer Subtimer Name', 'dt'])
open_chart = open_chart.sort_index(axis=0)
open_chart['moving_avg'] = open_chart['ATRT'].rolling(30).mean()
atrt_df['dt'] = pd.to_datetime(atrt_df['dt'].dt.date)
atrt_high_trans = atrt_df[atrt_df['transaction_cnt'] > 1000]
atrt_high_trans.sort_values(by=['transaction_cnt'], ascending=False)
len(atrt_high_trans)
grp_high_trans = atrt_high_trans.groupby(["Timer Subtimer Name", "dt"])
grp_high_trans.head()
open_chart_post.groupby('Timer Subtimer Name').nunique()
len(open_chart_post['Timer Subtimer Name'].unique())
len(open_chart_post)
open_chart_slim = open_chart_post[['ATRT', 'dt']]
open_chart_slim.set_index('dt')
open_chart_slim.sort_values('dt')

Stuff that didn't work
#atrt_df_PC = atrt_df_PC[['Timer Subtimer Name', 'dt', 'ATRT', 'application_name']]
#atrt_df_PC['mavg30'] = atrt_df_PC['ATRT'].rolling(30).mean()
#PC = atrt_df_PC.groupby(['Timer Subtimer Name', 'dt'])
#atrt_df_PC = atrt_df_PC.set_index(['Timer Subtimer Name', 'dt'])
#PC = atrt_df_PC
#PC.head()
#atrt_df_PC = atrt_df_PC.set_index(['Timer Subtimer Name'])
#atrt_df_PC = atrt_df_PC.sort_index(axis=0)
#atrt_df_PC = atrt_df_PC.groupby(['Timer Subtimer Name'])
#atrt_df_PC = atrt_df_PC.sort(['Timer Subtimer Name', 'dt']).groupby('Timer Subtimer Name')
#atrt_df_PC['mavg30'] = atrt_df_PC['ATRT'].rolling(30).mean()
#atrt_df_PC[0:100]```


Getting csv from website example
```
import csv
import urllib3
import requests

#This URL will be the URL that your login form points to with the "action" tag.
post_login_url = 'https://cernercare.com/accounts/login?returnTo=https%3A%2F%2Flightson.cerner.com%2Fsocial-auth%2Fcomplete%2Fprofessional%2F%3Fjanrain_nonce%3D2018-10-23T15%253A00%253A41ZYe3mOd'

#This URL is the page you actually want to pull down with requests.
request_url = 'https://lightson.cerner.com/clients/CHP_IN/domains/P1558/kpi/response-time/worst-timers.csv?dt=2018-10-22'
#request_url = 'https://lightson.cerner.com/api/metrics/trend.csv?category_name=Performance&metrics%5B%5D=RT_2SEC&cdr_ids=95370&data_type=ENVIRONMENT&date_type=DAILY&days_of_week=1%2C2%2C3%2C4%2C5%2C6%2C7&doc_cuid=LONfa31c997c3d543768036e2941&end_date=2018-10-22&physician_rollup_flag=2&start_date=2018-07-25'

payload = {
    'username': 'khickman1',
    'pass': '<password>'
}

with requests.Session() as session:
    post = session.post(post_login_url, data=payload)
    r = session.get(request_url)
    decoded_content = r.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list: 
        print(row)

for row in cr:
    with open("daily.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerows(row)
```
